Analysis reads in title and abstract data from file, runs mordecai and returns a data frame with unique locations for each entry

*Note* is it posisble to save as an sqlite file instead?

In [1]:
# Set up mordecai geoparser
from mordecai import Geoparser
geo = Geoparser()

Models path: C:\Users\deviv\anaconda3\envs\mordecai_env_3.8\lib\site-packages\mordecai\models\


In [2]:
# Import other packages
import pandas as pd
import pickle
import os
import sqlite3

In [7]:
## Read in geoparsing data
df = pd.read_csv('C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\data\\raw-data\\test-geocode-text.csv')

# combine title and abstract text into one variable
df['text'] = df['title'] + df['abstract']

# summarise
df.head()
df.shape

(100, 6)

In [10]:
## Read in the relevance predictions
relPredCon = sqlite3.connect("\\".join(["C:","Users","deviv","Python-working-folder","test-mordecai","data","raw-data","sql-databases","relevance-predictions.sqlite"]))

relPred_df = pd.read_sql_query("SELECT * from predRel", relPredCon)

# if needed, check table names:
#cursor = relPredCon.cursor()
#cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cursor.fetchall())

relPredCon.close()
relPred_df.head()

,analysis_id,relevance_mean,relevance_std,relevance_lower,relevance_upper
0,353882.0,0.025407,0.028486,0.000000,0.053893
1,53630.0,0.134056,0.226679,0.000000,0.360735
2,48146.0,0.009870,0.007417,0.002453,0.017286
3,370725.0,0.193188,0.188347,0.004841,0.381535
4,161800.0,0.092730,0.201549,0.000000,0.294279


In [13]:
## Read in the metadata for all the unique references
uniqueRefsCon = sqlite3.connect("\\".join(["C:","Users","deviv","Python-working-folder","test-mordecai","data","raw-data","sql-databases","unique-refs_v2.sqlite"]))

## add relevance predictions as a table to the new database
#relPred_df.to_sql("predRel", uniqueRefsCon, if_exists="replace")

# print 
uniqueRefs_df = pd.read_sql_query("SELECT analysis_id, duplicate_id, title, abstract FROM uniquerefs", uniqueRefsCon)


## close connections
uniqueRefsCon.close()

uniqueRefs_df.head()

,analysis_id,duplicate_id,title,abstract
0,296164,1913.2039,On the Precipitation of Calcium Carbonate in t...,None
1,296163,1917.2039,Climatic change and agricultural exhaustion as...,"I. Decline in Roman agriculture, 173. Contrast..."
2,384729,1917.2040,Tidal energy dissipation,None
3,384728,1920.2039,Tidal power,None
4,384727,1921.2039,Tidal power [3],None


In [14]:
## subset relevance predctions to >= 0.5
relPred_df = relPred_df[0.5 <= relPred_df['relevance_mean']]
len(relPred_df)

73928

In [20]:
## JOIN the relevance predictions with the text data
df = relPred_df.join(uniqueRefs_df, on="analysis_id", how = "inner", lsuffix = "_rel", rsuffix = "_refs")
df.head()

,analysis_id_rel,relevance_mean,relevance_std,relevance_lower,relevance_upper,analysis_id_refs,duplicate_id,title,abstract
7,230304.0,0.987904,0.005010,0.982894,0.992913,386513.0,2021.18186,Power generation performance of a pilot-scale ...,Reverse electrodialysis (RED) is a promising p...
8,311856.0,0.924882,0.172907,0.751975,1.000000,NaN,NaN,NaN,NaN
9,354126.0,0.967163,0.024341,0.942821,0.991504,NaN,NaN,NaN,NaN
18,39779.0,0.625630,0.303433,0.322197,0.929063,241024.0,2003.2600,"Weather, ice, and snow conditions at Deception...",An autonomous weather station (Terrestrial sta...
19,61009.0,0.989180,0.004243,0.984937,0.993422,363491.0,2007.7570,Reliability analysis for wind turbines,"Modern wind turbines are complex aerodynamic, ..."


In [19]:
df.shape # also get the dimensions -- for some reason I don't have matches for all the relevance predictions??

(36477, 10)

In [21]:
## Create another column with title and abstract combined to Geoparse
df['text'] = df['title'] + df['abstract']

# replace "the United States" with "United States" to avoid returning the virgin islands
df['text'] = df.text.replace("the United States", "United States", regex=True)

In [ ]:
## Geoparse the text and add results as a new column "geoparse"
df["geoparse"] = [geo.geoparse(x) for x in df["text"].astype('str')]

In [9]:
## Format dataframe

# create new formatted dataframe: df_clean
def flat_df(df):
    df_geo = df[df["geoparse"].str.len() != 0] #subset where geoparse string is not empty
    df_geo = df_geo.explode('geoparse') #Transforms each element of a list to a row and replicates index and all other columns. When more than one place name appears it creates more than one row.
    df_geo = pd.concat([df_geo.drop(['geoparse'], axis=1), df_geo['geoparse'].apply(pd.Series)], axis=1) #Extract from dic
    df_geo = pd.concat([df_geo.drop(['geo'], axis=1), df_geo['geo'].apply(pd.Series)], axis=1)
    df_geo = df_geo[df_geo['lat'].notnull()] #Removing empty latitude rows
    df_geo.lat = df_geo.lat.astype(float) #Transforms to float
    df_geo.lon =df_geo.lon.astype(float) #Transforms to float
    return df_geo
    
df_clean = flat_df(df)
print(df_clean.dtypes)
print(df_clean[0:5])
df_clean.head(5)

Unnamed: 0             int64
analysis_id            int64
duplicate_id         float64
title                 object
abstract              object
text                  object
word                  object
spans                 object
country_predicted     object
country_conf         float32
admin1                object
lat                  float64
lon                  float64
country_code3         object
geonameid             object
place_name            object
feature_class         object
feature_code          object
0                    float64
dtype: object
   Unnamed: 0  analysis_id  duplicate_id  \
1           2        77239     2004.2448   
3           4       141394     2017.7856   
3           4       141394     2017.7856   
3           4       141394     2017.7856   
4           5       172949     2012.3571   

                                               title  \
1  Organochlorine contaminants in sea turtles: Co...   
3  New resource for population genetics studies o...   
3 

,Unnamed: 0,analysis_id,duplicate_id,title,abstract,text,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,0
1,2,77239,2004.2448,Organochlorine contaminants in sea turtles: Co...,Monitoring toxic organochlorine (OC) compounds...,Organochlorine contaminants in sea turtles: Co...,Kemp,"[{'start': 845, 'end': 849}]",USA,0.879922,Texas,32.44264,-96.22998,USA,4702914,Kemp,P,PPL,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,New South Wales,"[{'start': 856, 'end': 871}]",AUS,0.906452,New South Wales,-33.00000,146.00000,AUS,2155400,State of New South Wales,A,ADM1,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,Australia,"[{'start': 873, 'end': 882}]",AUS,0.951695,NA,-25.00000,135.00000,AUS,2077456,Commonwealth of Australia,A,PCLI,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,Hardy-Weinberg,"[{'start': 1146, 'end': 1160}]",USA,0.852964,Pennsylvania,41.40639,-75.65667,USA,7127434,Harry and Jeanette Weinberg Memorial Library,S,LIBR,NaN
4,5,172949,2012.3571,Long-term trends of coral imports into the Uni...,The international trade in corals used to be p...,Long-term trends of coral imports into the Uni...,United States,"[{'start': 43, 'end': 56}]",VIR,0.879922,NA,18.34829,-64.98348,VIR,4796775,Virgin Islands of the United States,A,PCLD,NaN


In [ ]:
## Save as a .csv
df_clean.to_csv('C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\outputs\\geoparsed-records.csv', index=False)

In [ ]:
## Save as an sqlite database
database = 'C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\outputs\\geoparsed-records.sqlite'
conn = sqlite3.connect(database)
df_clean.to_sql("geoparsed-records", con=conn)